Example

In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['../src', '../example'])

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux


In [19]:
import pandas as pd
from hydroroot import radius
from hydroroot.main import hydroroot_flow
from hydroroot.init_parameter import Parameters
from hydroroot.generator.measured_root import mtg_from_aqua_data
from shared_functions import plot, read_archi_data, root_builder

In [20]:
parameter = Parameters()
parameter.read_file('parameters.yml')

In [21]:
fname = parameter.archi['input_dir'] + parameter.archi['input_file'][0]
df = read_archi_data(fname) # replace 3 lines in example_parameter_class.ipynb

In [22]:
# building the MTG from the dataframe df
# the output is the mtg, the primary root length, the total length and surface of the root, and the seed for the case of generated root here unsed
g, primary_length, total_length, surface, seed = root_builder(df = df, segment_length = parameter.archi['segment_length'],
            order_decrease_factor = parameter.archi['order_decrease_factor'], ref_radius = parameter.archi['ref_radius'])

In [23]:
axial_data = parameter.hydro['axial_conductance_data']
k0 = parameter.hydro['k0']
radial_data = ([0.0,0.2], [k0,k0])
g, Keq, Jv = hydroroot_flow(g, segment_length = parameter.archi['segment_length'], psi_e = parameter.exp['psi_e'], 
                            psi_base = parameter.exp['psi_base'], axial_conductivity_data = axial_data, 
                            radial_conductivity_data = radial_data)

In [27]:
k0_old = k0
F_old = (Jv - parameter.exp['Jv'])**2.0 # the objective function
k0 *= 0.9 # to initiate a simulation in the loop to compare with the previous one
eps = 1e-9 # the accuracy wanted
F = 1. # to launch the loop
# Newton-Raphson loop to get k0
while (F > eps):
    radial_data = ([0.0,0.2], [k0,k0])
    g, Keq, Jv = hydroroot_flow(g, segment_length = parameter.archi['segment_length'], psi_e = parameter.exp['psi_e'], 
                                psi_base = parameter.exp['psi_base'], axial_conductivity_data = axial_data, 
                                radial_conductivity_data = radial_data)

    F = (Jv - parameter.exp['Jv']) ** 2.0 # the objective function

    if abs(F) > eps:
        dfdk0 = (F - F_old) / (k0 - k0_old) # the derivative of F according to k0

        k0_old = k0

        k0 = k0_old - F / dfdk0 # new estimate
        while k0 < 1.0e-3:
            k0 = 0.5 * k0_old

        F_old = F

In [25]:
print('experimental Jv: ', parameter.exp['Jv'], 'simulated Jv: ', Jv, 'adjusted k: ', k0)

experimental Jv:  0.00452 simulated Jv:  0.004545766780791878 adjusted k:  33.93466828329859
